In [ ]:
# %pip install scikit-learn
# %pip install pandas

In [18]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import pandas as pd
import time, concurrent.futures, json, requests, re
import numpy as np
import networkx as nx
from threading import Lock

In [ ]:
cosine_similarity([[3,1]],[[6,2]])
cosine_distances([[3,1]],[[6,2]])

In [ ]:
cosine_similarity([[3,0]],[[0,8]])

In [ ]:
cosine_similarity([[3,1]],[[3,2]])

In [ ]:
doc1 = """
iphone sales contributed to 70% of revenue. iphone demand is increasing by 20% yoy. 
the main competitor phone galaxy recorded 5% less growth compared to iphone" ipad ipad 
"""

doc2 = """
The upside pressure on volumes for the iPhone 12 series, historical outperformance 
in the July-September time period heading into launch event, and further catalysts in relation
to outperformance for iPhone 13 volumes relative to lowered investor expectations implies a 
very attractive set up for the shares.
"""

doc3 = """
samsung's flagship product galaxy is able to penetrate more into asian markets compared to
iphone. galaxy is redesigned with new look that appeals young demographics. 60% of samsung revenues
are coming from galaxy phone sales
"""

doc4 = """
Samsung Electronics unveils its Galaxy S21 flagship, with modest spec improvements 
and a significantly lower price point. Galaxy S21 price is lower by ~20% (much like the iPhone 12A), 
which highlights Samsung's focus on boosting shipments and regaining market share.
"""

In [ ]:
df = pd.DataFrame([
        {'iPhone': 3,'galaxy': 1, 'ipad': 4, 'hi': 0},
        {'iPhone': 2,'galaxy': 0, 'ipad': 1, 'hi': 0},
        {'iPhone': 1,'galaxy': 3, 'ipad': 0, 'hi': 0},
        {'iPhone': 1,'galaxy': 2, 'ipad': 2, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
    ],
    index=[
        "doc1",
        "doc2",
        "doc3",
        "doc4",
        "doc5","doc6","doc7","doc8"
    ])

In [ ]:
df
#df.loc["doc1":"doc1"]

In [ ]:
cosine_similarity(df.loc["doc1":"doc1"],df.loc["doc5":"doc5"])

In [ ]:
data =  ["independent","abruptness","abandonment"]
response_API = requests.post("http://127.0.0.1:5000/searchdata", json = {"word": "carvet"})

sendData = response_API.text
print(sendData)

In [19]:
# request to get books
def getBooksThread(bookId, timeout=10):
    response_API = requests.get('https://gutendex.com/books/{}'.format(bookId), timeout=timeout)
    data = response_API.text
    parse_json = json.loads(data)
    if parse_json.get('detail') != None:
        # print(bookId)
        return 'NOT_FOUND'
    return parse_json 

def getBooksData(listBooks):
    print("Running get books threads:")
    threaded_start = time.time()
    booksData = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for bookId in listBooks:
            futures.append(executor.submit(getBooksThread, bookId))
        for future in concurrent.futures.as_completed(futures):
            if future.result() != 'NOT_FOUND':
                booksData.append(future.result())
            # print(future.result())
    print("Threaded get books", time.time() - threaded_start)
    return booksData



def getListBooks(listBooks):
    def transformData(d):
        res = []
        if d.get('formats')!=None:
            for t in d['formats'].keys():
                checkEnd = d['formats'][t].split('.').pop()
                if checkEnd == 'txt':
                    res.append({
                        'id': d['id'],
                        'text_url': d['formats'][t]
                    })
        return res
    
    data = getBooksData(listBooks)
    result = []
    
    print("Running get list books:")
    threaded_start = time.time()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for d in data:
            futures.append(executor.submit(transformData, d))
        for future in concurrent.futures.as_completed(futures):
            result += future.result()
    print("Threaded get list books time:", time.time() - threaded_start)
    return result

def getTableIndex(listBooks):
    tableIndex = dict()
    booksInfo = []
    listBooksData = getListBooks(listBooks)

    lock = Lock()

    def readBook(book):
        response_API = requests.get(book['text_url'])
        data = response_API.text
        #### Option 1: Prendre seulement des mots avec carateres de 4 à 10
        words = re.findall(r"[A-Za-z]{4,10}\w+", data)
        occurentCounts = dict()

        for word in words:
            w = word.lower()
            lock.acquire()
            # Count for table index all books
            if w in tableIndex:
                if book['id'] in tableIndex[w]:
                    tableIndex[w][book['id']] += 1
                else:
                    tableIndex[w][book['id']] = 1
            else:
                tableIndex[w] = dict({book['id']: 1})

            # Count for table index for each book
            if w in occurentCounts:
                occurentCounts[w] += 1
            else:
                occurentCounts[w] = 1
            lock.release()

        return {
            "bookId": book['id'],
            "words": occurentCounts,
            "totalWords": len(words),
            "totalWordsWithOccur": len(occurentCounts)
        }
    print("Running table index:")
    threaded_start = time.time()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for book in listBooksData:
            futures.append(executor.submit(readBook, book))
        for future in concurrent.futures.as_completed(futures):
            booksInfo.append(future.result())
    print("Threaded table index:", time.time() - threaded_start)
    return tableIndex, booksInfo



In [20]:

listBooks = [49345,56667,1,2,3,4,5,6,7]
tableIndexData, booksInfo = getTableIndex(listBooks)
historyWords =dict({
  "carver": 1, 
  "carvet": 1, 
  "saigon": 2, 
  "sargon": 3
})

print(tableIndexData['sargon'])


Running get books threads:
Threaded get books 0.49893784523010254
Running get list books:
Threaded get list books time: 0.0011250972747802734
Running table index:
Threaded table index: 1.6427960395812988
{56667: 27, 49345: 98}


In [25]:

def toTableCosine(tableIndexData):
    booksData = dict()
    for word in tableIndexData:
        for b in tableIndexData[word].keys(): 
            if b in booksData:
                booksData[b].update(dict({word:tableIndexData[word][b]}))
            else:
                booksData.update(dict({b: dict({word:tableIndexData[word][b]})}))

    return booksData

def cosineSearchWord(historyWords, tableIndexData):
    # Init variable
    result = dict()
    booksData = dict({'history':historyWords})
    for word in historyWords.keys():
        if word in tableIndexData:
            for b in tableIndexData[word].keys(): 
                if b in booksData:
                    booksData[b].update(dict({word:tableIndexData[word][b]}))
                else:
                    booksData.update(dict({b: dict({word:tableIndexData[word][b]})}))
    bookDF = pd.DataFrame(booksData.values(),
        index=booksData.keys()).fillna(0)
    for cs in list(booksData.keys())[1:]:
        result[cs] = cosine_similarity(bookDF.loc["history":"history"],bookDF.loc[cs:cs])[0][0]
    return result

def getMatrixCloseness(tableIndexData):
    # Init variable
    booksData = dict()
    for word in tableIndexData:
        for b in tableIndexData[word].keys(): 
            if b in booksData:
                booksData[b].update(dict({word:tableIndexData[word][b]}))
            else:
                booksData.update(dict({b: dict({word:tableIndexData[word][b]})}))
    bookDF = pd.DataFrame(booksData.values(),
        index=booksData.keys()).fillna(0)
    # print(bookDF)
    matrixCloseness = []
    for b1 in list(booksData.keys()):
        for b2 in list(booksData.keys()):
            if b1 != b2:
                res = cosine_similarity(bookDF.loc[b1:b1],bookDF.loc[b2:b2])[0][0]
                if res*100 > 50: # > 50% -> add edge
                    matrixCloseness.append((b1,b2))
    
    print(matrixCloseness)
    # Create the graph representing the reading app
    G = nx.Graph()
    G.add_edges_from(matrixCloseness)
    closenessData = []

    # Compute the closeness centrality of each node in the graph
    closeness_centrality = nx.closeness_centrality(G)

    # Print the closeness centrality of each node
    for node, closeness in closeness_centrality.items():
        closenessData.append({"bookId": node, "closeness":closeness })

    sortedClosenessData = sorted(closenessData, key=lambda d: d['closeness'], reverse=True) 
    return sortedClosenessData

searchedBook = cosineSearchWord(historyWords, tableIndexData)
print(searchedBook)
matC = getMatrixCloseness(tableIndexData)
for i in matC:
    print (i)

# getMatrixCloseness(tableIndexData)


{1: 0.2581988897471611, 7: 0.2581988897471611, 49345: 0.7798254364032349, 56667: 0.7745966692414834}
[(4, 6), (4, 2), (4, 3), (4, 7), (6, 4), (6, 2), (6, 3), (6, 1), (6, 7), (2, 4), (2, 6), (2, 3), (2, 1), (2, 5), (2, 7), (3, 4), (3, 6), (3, 2), (3, 1), (3, 7), (1, 6), (1, 2), (1, 3), (1, 5), (5, 2), (5, 1), (7, 4), (7, 6), (7, 2), (7, 3), (49345, 56667), (56667, 49345)]
{'bookId': 2, 'closeness': 0.75}
{'bookId': 6, 'closeness': 0.6428571428571428}
{'bookId': 3, 'closeness': 0.6428571428571428}
{'bookId': 4, 'closeness': 0.5625}
{'bookId': 7, 'closeness': 0.5625}
{'bookId': 1, 'closeness': 0.5625}
{'bookId': 5, 'closeness': 0.44999999999999996}
{'bookId': 49345, 'closeness': 0.125}
{'bookId': 56667, 'closeness': 0.125}


In [ ]:
booksData = []
for book in booksInfo:
    count = 0
    for w in historyWords.keys():
        if w in book['words']:
            # count number occurs in document
            # count += book['words'][w]
            # count if have that word in ducument
            count += historyWords[w]
    booksData.append({
        "bookId": book['bookId'],
        "jaccard": (count/book['totalWordsWithOccur'])
    })

[print(b) for b in booksData]

In [ ]:
data = {
  "1": 0.6, 
  "6": 0.4472135954999579, 
  "7": 0.4472135954999579, 
  "49345": 0.4517534811203126, 
  "56667": 0.6439041435846341
}

sortedBooks = dict(sorted(data.items(),key=lambda x:x[1], reverse=True))

print(sortedBooks)

In [12]:


data = [
    {  "bookId": 6,  "jaccard": 0.001 }, 
    { "bookId": 1,  "jaccard": 0.00011661807580174928 }, 
    { "bookId": 49345, "jaccard": 8.2223318533136e-05}, 
    {  "bookId": 56667,  "jaccard": 5.6507816914673195e-05}, 
    { "bookId": 3,  "jaccard": 0.0 }, 
    {  "bookId": 5, "jaccard": 0.0 }, 
    {
        "bookId": 4, 
        "jaccard": 0.0
    }, 
    {
        "bookId": 2,
        "jaccard": 0.0
    }, 
    {
        "bookId": 7,
        "jaccard": 0.0
    }
]

# print(data)
sorted_list = sorted(data, key=lambda x: x['jaccard'], reverse=True)
print(sorted_list)


[{'bookId': 6, 'jaccard': 0.001}, {'bookId': 1, 'jaccard': 0.00011661807580174928}, {'bookId': 49345, 'jaccard': 8.2223318533136e-05}, {'bookId': 56667, 'jaccard': 5.6507816914673195e-05}, {'bookId': 3, 'jaccard': 0.0}, {'bookId': 5, 'jaccard': 0.0}, {'bookId': 4, 'jaccard': 0.0}, {'bookId': 2, 'jaccard': 0.0}, {'bookId': 7, 'jaccard': 0.0}]


In [17]:
print(1 < 8.2223318533136e-05)

False
